In [1]:
import glob
import os
import math
import matplotlib.pyplot as plt
import numpy as np
import re
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.applications.vgg16 import VGG16
#from cv2 import imshow

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.optimizers.legacy import Adam

2023-06-07 22:45:46.490194: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 22:45:46.931578: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-07 22:45:46.931634: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-07 22:45:48.387272: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Data pre-processing

In [2]:
root_dir = '/home/haohao/Desktop/Hieu/VisualPhish/'
dataset_dir_benign = root_dir + 'benign_test'
dataset_dir_phishing = root_dir + 'phishing'
dataset_dir_trusted = root_dir + 'trusted_list'
benign = list(glob.glob(dataset_dir_benign + '/*'))
phishing = list(glob.glob(dataset_dir_phishing + '/**/*'))
trusted_list = list(glob.glob(dataset_dir_trusted + '/**/*'))

cnt = 0
for f in benign:
  if f.upper().find(".PNG") == -1 and f.upper().find(".JPG") == -1:
    try:
      os.remove(f)
      cnt += 1
    except OSError as e:
      print(e)

print("benign removed: " + str(cnt))
cnt = 0

for f in phishing:
  if f.upper().find(".PNG") == -1 and f.upper().find(".JPG") == -1:
    try:
      os.remove(f)
      cnt += 1
    except OSError as e:
      print(e)

print("phishing removed: " + str(cnt))
cnt = 0

for f in trusted_list:
  if f.upper().find(".PNG") == -1 and f.upper().find(".JPG") == -1:
    try:
      os.remove(f)
      cnt += 1
    except OSError as e:
      print(e)

print("trusted_list removed: " + str(cnt))

print('Number of benign images: ' + str(len(benign)))
print(benign[:5])
print('Number of phishing images: ' + str(len(phishing)))
print(phishing[:5])
print('Number of trusted images: ' + str(len(trusted_list)))
print(trusted_list[:5])

benign removed: 0
phishing removed: 0
trusted_list removed: 0
Number of benign images: 3107
['/home/haohao/Desktop/Hieu/VisualPhish/benign_test/02673.png', '/home/haohao/Desktop/Hieu/VisualPhish/benign_test/02357.png', '/home/haohao/Desktop/Hieu/VisualPhish/benign_test/00993.PNG', '/home/haohao/Desktop/Hieu/VisualPhish/benign_test/00340.png', '/home/haohao/Desktop/Hieu/VisualPhish/benign_test/01948.png']
Number of phishing images: 1195
['/home/haohao/Desktop/Hieu/VisualPhish/phishing/docmagic/T52_0.png', '/home/haohao/Desktop/Hieu/VisualPhish/phishing/allegro/T6_8.png', '/home/haohao/Desktop/Hieu/VisualPhish/phishing/allegro/T6_1.png', '/home/haohao/Desktop/Hieu/VisualPhish/phishing/allegro/T6_6.png', '/home/haohao/Desktop/Hieu/VisualPhish/phishing/allegro/T6_3.png']
Number of trusted images: 1870
['/home/haohao/Desktop/Hieu/VisualPhish/trusted_list/twitter/T136_71.png', '/home/haohao/Desktop/Hieu/VisualPhish/trusted_list/twitter/T136_85.png', '/home/haohao/Desktop/Hieu/VisualPhish/tru

# Data balancing

In [3]:
saved_dir = 'unused_trusted_list'
saved_path = os.path.join(root_dir, saved_dir)
if not os.path.exists(saved_path):
    os.mkdir(saved_path)

### Move images out of dataset

In [56]:
files = list(glob.glob(dataset_dir_trusted + '/**/*'))
print(files[:5])

['/home/haohao/Desktop/Hieu/VisualPhish/trusted_list/docmagic/T52_42.png', '/home/haohao/Desktop/Hieu/VisualPhish/trusted_list/docmagic/T52_48.png', '/home/haohao/Desktop/Hieu/VisualPhish/trusted_list/docmagic/T52_26.png', '/home/haohao/Desktop/Hieu/VisualPhish/trusted_list/docmagic/T52_32.png', '/home/haohao/Desktop/Hieu/VisualPhish/trusted_list/docmagic/T52_21.png']


In [57]:
movingRatio = 0.8

numOfMovedFiles = math.floor(movingRatio * len(files))

# We can pick images randomly here

for f in files[:numOfMovedFiles]:
    os.rename(f, saved_path + '/' + f.replace('/', '----------'))

### Move back

In [53]:
files = list(glob.glob(os.path.join(root_dir, saved_dir) + '/*'))

for f in files:
    originalFilename = f.replace(os.path.join(root_dir, saved_dir) + '/', '').replace('----------', '/')
    os.rename(f, originalFilename)
  

# Load dataset and normalization

In [4]:
img_height = 256
img_width = 256
phishing_size = len(phishing)
trusted_size = len(trusted_list)

In [5]:
# We can use trusted_list with phishing labels

dataseta_train = tf.keras.preprocessing.image_dataset_from_directory(dataset_dir_phishing, validation_split=0.3, subset='training', seed=123, image_size=(img_height, img_width), batch_size=None, labels=[1 for i in range(phishing_size)], label_mode='int')
datasetb_train = tf.keras.preprocessing.image_dataset_from_directory(dataset_dir_benign, validation_split=0.3, subset='training', seed=123, image_size=(img_height, img_width), batch_size=None, labels=None)
datasetc_train = tf.keras.preprocessing.image_dataset_from_directory(dataset_dir_trusted, validation_split=0.3, subset='training', seed=123, image_size=(img_height, img_width), batch_size=None, labels=[1 for i in range(trusted_size)], label_mode='int')

dataseta_train = dataseta_train.map(lambda x, y: (x, tf.cast(y, tf.float32)))
datasetb_train = datasetb_train.map(lambda x: (x, tf.constant(0.0)))
datasetc_train = datasetc_train.map(lambda x, y: (x, tf.cast(y, tf.float32)))

dataseta_test = tf.keras.preprocessing.image_dataset_from_directory(dataset_dir_phishing, validation_split=0.3, subset='validation', seed=123, image_size=(img_height, img_width), batch_size=None, labels=[1 for i in range(phishing_size)], label_mode='int')
datasetb_test = tf.keras.preprocessing.image_dataset_from_directory(dataset_dir_benign, validation_split=0.3, subset='validation', seed=123, image_size=(img_height, img_width), batch_size=None, labels=None)
datasetc_test = tf.keras.preprocessing.image_dataset_from_directory(dataset_dir_trusted, validation_split=0.3, subset='validation', seed=123, image_size=(img_height, img_width), batch_size=None, labels=[1 for i in range(trusted_size)], label_mode='int')

dataseta_test = dataseta_test.map(lambda x, y: (x, tf.cast(y, tf.float32)))
datasetb_test = datasetb_test.map(lambda x: (x, tf.constant(0.0)))
datasetc_test = datasetc_test.map(lambda x, y: (x, tf.cast(y, tf.float32)))

Found 1195 files belonging to 155 classes.
Using 837 files for training.


2023-06-07 22:46:05.188502: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-07 22:46:05.188551: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-07 22:46:05.188596: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (haohao-virtual-machine): /proc/driver/nvidia/version does not exist
2023-06-07 22:46:05.190789: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 3107 files belonging to 1 classes.
Using 2175 files for training.
Found 1870 files belonging to 155 classes.
Using 1309 files for training.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Found 1195 files belonging to 155 classes.
Using 358 files for validation.
Found 3107 files belonging to 1 classes.
Using 932 files for validation.
Found 1870 files belonging to 155 classes.
Using 561 files for validation.


In [6]:
train_ds = dataseta_train.concatenate(datasetb_train).concatenate(datasetc_train)
test_ds = dataseta_test.concatenate(datasetb_test).concatenate(datasetc_test)

In [7]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [8]:
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Create model

In [9]:
def create_model(input_shape, optimizer='rmsprop', fine_tune=0):
  conv_base = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
  if fine_tune > 0:
      for layer in conv_base.layers[:-fine_tune]:
          layer.trainable = False
  else:
      for layer in conv_base.layers:
          layer.trainable = False

  top_model = conv_base.output

  top_model = Flatten(name="flatten")(top_model)
  top_model = Dense(4096, activation='relu')(top_model)
  top_model = Dense(1072, activation='relu')(top_model)
  top_model = Dropout(0.2)(top_model)
  output_layer = Dense(1, activation='sigmoid')(top_model)
  
  model = Model(inputs=conv_base.input, outputs=output_layer)

  model.compile(optimizer=optimizer, 
                loss='binary_crossentropy',
                metrics=['accuracy'])
  
  return model


In [10]:
#train_ds = train_ds.take(330)
#test_ds = test_ds.take(330)


# Set up N parties

In [10]:
# Split dataset into n parties
numOfParties = 5
n_parties_train_set = []
n_parties_test_set = []

# number of dataset examples each party
local_num_examples = []

for i in range(numOfParties):
  n_parties_train_set.append(train_ds.shard(numOfParties, i))
  n_parties_test_set.append(test_ds.shard(numOfParties, i))
  local_num_examples.append(tf.data.experimental.cardinality(n_parties_train_set[i]).numpy())


# Federated Learning

In [11]:
input_shape = (img_height, img_width, 3)
optim_1 = Adam(learning_rate=0.001)
n_epochs = 1
fl_round = 5
batch_size = 32

In [12]:
for i in range(numOfParties):
  n_parties_train_set[i] = n_parties_train_set[i].batch(batch_size)
  n_parties_test_set[i] = n_parties_test_set[i].batch(batch_size)

In [13]:
global_model = create_model(input_shape, optim_1, fine_tune=0)

totNumOfExamples = sum(local_num_examples)

for round in range(fl_round):
  print("#Round: " + str(round))
  # model weights of n parties
  local_weights = []

  for i in range(numOfParties):
    print("Client: " + str(i))
    local_model = create_model(input_shape, optim_1, fine_tune=0)
    local_model.set_weights(global_model.get_weights())

    # train locally
    local_model.fit(n_parties_train_set[i], epochs=n_epochs, validation_data=n_parties_test_set[i], verbose=1)
    local_weights.append(local_model.get_weights())

  # update global model with mean of weights
  avg_weights = []
  for j in range(len(local_weights[0])):  # for each layer in the model
      layer_weights = [local_weights[i][j] for i in range(len(local_weights))]  # collect this layer's weights from each client
      layer_weights_avg = np.average(layer_weights, axis=0, weights=[num / totNumOfExamples for num in local_num_examples])  # compute the weighted average
      avg_weights.append(layer_weights_avg)  # append the average weights for this layer

  global_model.set_weights(avg_weights)

#Round: 0
Client: 0
 4/28 [===>..........................] - ETA: 5:59 - loss: 0.2412 - accuracy: 0.7969

2023-06-07 22:48:33.119882: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 219 of 1024
2023-06-07 22:48:43.297103: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 437 of 1024


 5/28 [====>.........................] - ETA: 6:21 - loss: 0.1929 - accuracy: 0.8375

2023-06-07 22:48:53.083189: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:392] Filling up shuffle buffer (this may take a while): 722 of 1024
2023-06-07 22:49:03.046509: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


 8/28 [=======>......................] - ETA: 7:24 - loss: 226.0508 - accuracy: 0.5547

In [ ]:
global_model.save(root_dir + "model.h5")

# Evaluation

In [ ]:
vgg_model = keras.models.load_model(root_dir + 'model.h5')

In [ ]:
predictions = []
y_true = []
for x, y in test_ds:
  predictions.append(vgg_model.predict(x))
  y_true.append(y)


In [ ]:
print(predictions[:10])
print(y_true[:10])
print(len(predictions))
print(len(y_true))

In [ ]:
iter = 59 # ceil(samples/batch_size)
y_pred = []
for i in range(iter):
  for p in predictions[i]:
    if p[0] >= 0.5:
      y_pred.append(1)
    else:
      y_pred.append(0)

In [ ]:
y = []

for tensor in y_true:
  for v in tensor:
    if (v == 0):
      y.append(0)
    else:
      y.append(1)

In [ ]:
def evaluation(y_test, y_pred):
  result1 = classification_report(y_test, y_pred)
  print("Classification Report:",)
  print (result1)
  result2 = accuracy_score(y_test, y_pred)
  print("Accuracy:", result2)
  result3 = precision_score(y_test, y_pred)
  print("Precision:",result3)
  result4 = recall_score(y_test, y_pred)
  print("Recall:",result4)
  result5 = f1_score(y_test, y_pred)
  print("F1:",result5)
  result6 = roc_auc_score(y_test, y_pred)
  print("AUC:",result6)
  result7 = confusion_matrix(y_test, y_pred)
  print("Confusion matrix: \n", result7)

In [ ]:
evaluation(y, y_pred)

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.82      0.86       932
           1       0.84      0.91      0.87       936

    accuracy                           0.87      1868
   macro avg       0.87      0.87      0.87      1868
weighted avg       0.87      0.87      0.87      1868

Accuracy: 0.8661670235546038
Precision: 0.8382642998027613
Recall: 0.9081196581196581
F1: 0.8717948717948717
AUC: 0.8660769964418032
Confusion matrix: 
 [[768 164]
 [ 86 850]]


In [ ]:
print(y.count(0))
print(y.count(1))
print(y_pred.count(0))
print(y_pred.count(1))

932
936
854
1014


In [ ]:
plt.plot(vgg_history.history['loss'])
plt.plot(vgg_history.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(vgg_history.history['accuracy'])
plt.plot(vgg_history.history['val_accuracy'])
plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()